<a href="https://colab.research.google.com/github/averyrop/Spotify-ML-showcase/blob/main/Avery_Rich's_technical_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://www.richs.com/wp-content/uploads/2019/07/logo.png" alt="RPC" title="RPC" width="30%" height="100%"/>
#**Technical Assessment**

>[](#scrollTo=yPsyJayRo_uE)

>[Welcome!](#scrollTo=e2pnVwxqDl7b)

>>[Instructions](#scrollTo=Mn-pAU_2Fbab)

>>[Data](#scrollTo=G3IOsNqmV7Zx)

>>[Dependencies](#scrollTo=LCVN4ikGGdqt)

>>[Part 1: Exploratory Data Analysis](#scrollTo=swZ97WzCHWgb)

>>[Part 2: Data Preparation & Modeling](#scrollTo=ZHolP1FUmv2d)

>>[Part 3: Application](#scrollTo=wlOl3XZOY9xK)

>>[Wrap Up](#scrollTo=ddURXPpvZfeb)




# Welcome!

Welcome to the Technical Assessment portion of your interview for the Assoicate Data Scientist role. The goal of this assessment is to present you with a sample problem and evaluate your approach to solving it.  While we will give you a couple of days (2-3) to complete, it should not take more than a few hours.

Feel free to leverage this cloud environment or your own preferred environment to complete the assessment.

**Notes, if using Google Colab**: 
- If you decide to use Google Colab you will need to create a Google Account.
- You will need to save a copy of this notebook as you will not be able to edit this version directly.
- Google Colab Jupyter notebooks runs on the cloud and will reset every 12 hours. Your work will be automatically saved but you may have to run your code again when you return.

*This work will not give Rich Products any competitive advantage as this is a fake problem with fake data. This is truly an assessment to evaluate your technical and problem solving skills.*

---

## Instructions

**Business Problem**: Earlier in the year, we hired an external consulting partner to provide us with *loyalty scores* for our retail customers. Just so you are aware, the loyalty score measures the percentage of grocery spend that a customer allocates to our grocery store vs our competitors.

Unfortunately, the consultancy was only able to match around half of our customers to their loyalty database - so while we now have loyalty scores for those customers, we're missing scores for the other half.

We have a hypothesis that this loyalty score could be related to some customer metrics that we have in our database - but we don't know for sure. If we could find a way to predict the missing loyalty scores this would add huge value to our business as it would enable us to target all of our customers with specific orders and discounts based on their loyalty.

------

Be sure to explain your thought process as it pertains to your solution approach. Be *clear*, *concise* and, most importantly, ***creative***!


## Data

The data for this sample problem can be accessed [here](https://drive.google.com/file/d/1mQMIwMIxB-zfE75k2RRGf57BUbs_BxvG/view?usp=sharing). Please let us know if you have any issues accessing the data file.

## Dependencies

Add all your dependencies here so anyone can run your code later.

The details of this section don't really matter too much from a story telling standpoint, but here we import every function we'll need for later.  We have to upgrade one so it's capable of doing what we need and import others for manipulation, exploration, cleaning, and analysis of the data.

In [1]:
pip install openpyxl --upgrade --pre #Updated to use a function


Requirement already up-to-date: openpyxl in /usr/local/lib/python3.6/dist-packages (3.0.6)


In [2]:
from google.colab import drive

In [3]:
from openpyxl import load_workbook
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt

In [4]:
from sklearn import linear_model
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error

###Importing and organizing Data

In this section we need to import the actual data so we can work with it. I choose to do it by mounting my google drive so I can access the data directly from there, but this doesn't allow anyone running this to get it, since it would link to their drive, which may not have the data or have it in the same place. Mounting the drive requires copying an access code so it can work, so don't be alarmed by the link that may pop up.

In [5]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now we use the load_workbook function we imported to easily work with .xlsl files. I create a "workbook" from the excel file and take all the data out, transferring it to a python dictionary. It is a convenient way of maintaining all the relationships in the data while also allowing me to easily create pandas dataframes from the data later.

In order for this notebook to work the file Copy of tech_assessment_data.xlsx needs to be in your google drive. If it's not the data will not be there to use.

In [6]:
tech_data_book = load_workbook(filename = './drive/MyDrive/Copy of tech_assessment_data.xlsx') 

In [7]:
tech_data_dict = {}   #Empty ditionary to fill

for sheet in tech_data_book.sheetnames:     #Going through every sheet in the excel file
  tech_data_dict[sheet] = {}                #Creating a separate spot for each sheet in the dictionary
  header = True

  for row in tech_data_book[sheet].iter_rows(values_only=True):   #Going through every row of data in every sheet

    if header:                                                    #Taking the first row values as the column titles
      for val in row:
        tech_data_dict[sheet][val] = []                           #Creating a separate spot in every sheet for every column title
        header=False

    else:
      for i,key in enumerate(tech_data_dict[sheet].keys()):       #Preparing to input something into every space in the new spots for every column title
        if row[i] == None or row[i] == 'nan':
          tech_data_dict[sheet][key].append(np.nan)               #If we don't have data let python know by filling with "nan"
        else:
          tech_data_dict[sheet][key].append(row[i])               #If we do have data put it in



In [8]:
tech_frame_dict = {}    #Now I take the data from the dictionaries from each sheet and turn them into dataframes
                        # These dataframes are stored in a dictionary
for sheet in tech_data_dict.keys():

  tech_frame_dict[sheet] = pd.DataFrame(tech_data_dict[sheet])      #pandas (one of the libraries imported) makes it easy to turn dictionaries into dataframes
                                                                    #easily readable table like structures for data

In [9]:
database_info = tech_frame_dict.pop('database_info')        #One of the sheets didn't really have info for analysis, it was for reading, so I pull it from the rest of the data

**Note**: The following structure has been provided to assist you. However, feel free to leverage your own structure for tackling the problem above.

## Part 1: Exploratory Data Analysis

Use this section to do an in-depth analysis of the data. 

In this section I take a look at the data to figure out some stuff about it, like how big each dataframe is, which values are continuous and which are categorical, and what I should do about empty values where we didn't have data about something

Below we see that we have 5 dataframes full of data, all with relatively few attributes, or column headers. The biggest one by far has around 38000 rows, so this will probably be where a lot of important info can be pulled from. We don't have to worry about high dimensionality that could mess up algorithms, because we have for more rows of data then columns in most of these cases. One has only 5 rows, but upon inspection later we'll see that this isn't really a problem.

In [10]:
for df in tech_frame_dict:
  print(tech_frame_dict[df].shape)

(870, 6)
(38506, 7)
(5, 3)
(870, 5)
(400, 2)


Here I see which columns are categorical and continuous, because they'll need slightly different data preprocessing. I decide this based on how many unique values there are in each category. For example out of 900 customers if you see how many unique values for gender there are answered you'll only find M, F, or none. However if you were to check how many different "distance from stores" there are you'll find a lot, as 5 miles and 5.1 miles won't be identified as the same, and therefore every slightly different distance value will be counted and clue us in that it's continuous.

This method isn't foolproof, as a categorical value with a ton of different options can exist (e.g. a column for written in occupation could have a slightly different answer for everyone such as teacher, spanish teacher, music teacher, etc.) but it catches a lot and the ones that aren't caught will show their faces eventually. The sensitivity of this can also be tuned based on the fraction of unique values to total values that we call the cutoff threshold between categorical and continuous

In [11]:
diverse_dict = {}       #Create a dictionary to fill to organize the info
cat_list = []           #Start a list for categorical values
con_list = []           #start a list for continuous values

for df in tech_frame_dict:    #Go through every dataframe
  diverse_dict[df]={}         
  for col in tech_frame_dict[df]:   #Go through every attribute in every table

    val = (len(set(tech_frame_dict[df][col]))/len(tech_frame_dict[df][col]))    #Calculate the fraction of unique values to number of responses

    diverse_dict[df][col] = val                                                 #Input fraction into dictionary in proper spot

    if val < .01:
      cat_list.append(col)                                                      #Decide it's categorical if this fraction is under one percent
    else:
      con_list.append(col)                                                      #Decide it's continuous if this fraction is over one percent

Next we see which values are empty in our tables. We need to know this because a lot of ML algorithms don't like null values, so we need to either fill them or remove them, depending on what's appropriate. If the fraction of empty values to total values is low we can fill them, but if it's too high we may need to get rid of the column all together.

In [12]:
nan_dict = {}             #create empty dict for filling with the data on what percentage of null values every column of every table has

for df in tech_frame_dict:    #Going through every table
  nan_dict[df]=tech_frame_dict[df].isna().apply(lambda col: sum(col)/len(col))   #Using a lambda and pandas convenient isna to calculate the fraction of null values  

Below I print out the fractions of null values for every column. A lot of them have 0 null values, but distance from store, gender, and credit score have a few, though it's a very small fraction, less than 1 percent. We can see that email and phone have 100% null values so these are useless columns to us and will be dropped. We also have an empty one called NaN, but this was a mistake anyway, so this column will be dropped regardless.

In [13]:
for key in nan_dict:
  print(nan_dict[key])

customer_id            0.000000
distance_from_store    0.005747
gender                 0.005747
credit_score           0.009195
email                  1.000000
phone                  1.000000
dtype: float64
customer_id         0.0
transaction_date    0.0
transaction_id      0.0
product_area_id     0.0
num_items           0.0
sales_cost          0.0
NaN                 1.0
dtype: float64
product_area_id      0.0
product_area_name    0.0
profit_margin        0.0
dtype: float64
customer_id      0.0
campaign_name    0.0
campaign_date    0.0
mailer_type      0.0
signup_flag      0.0
dtype: float64
customer_id               0.0
customer_loyalty_score    0.0
dtype: float64


Below I start plotting histograms of the values that have some nulls to see how to fill them. Based on the distribution we'll decide how to fill. 

In [14]:
px.histogram(tech_frame_dict['customer_details'],x='distance_from_store',title='Distance from store count histogram') #Plotting a histogram of the distances from store

The distance from store histogram mostly has people living under 25 miles or whatever metric is being used from the store. There are some massive, but I will not remove the customers that are outliers in this sense. I make this choice because The lasso method I plan to implement is good at dealing with outliers, so it should be unaffected. Intuitively I'm saying that while the circumstances of people who live over 25 miles from the store are different from those within range, they may have other important info to gather from the other categories. (e.g. it might show that someone with a low credit score is super likely to be loyal, even if they live 400 miles away).

In terms of null values though, we can fill this them with the median, which will be where most people are.

In the interest of showing the data on a reasonable scale I will remove the outliers just to plot it, but the outliers will remain in the analysis. I said 25 miles above, but that was just an estimate. While I should technically be using interquartile ranges by convention, I will just show where the 99th percentile of this data for simplicity.

In [15]:
np.percentile(tech_frame_dict['customer_details']['distance_from_store'],99)
out_remove = tech_frame_dict['customer_details']['distance_from_store'][tech_frame_dict['customer_details']['distance_from_store']<6.53]

The 99th percentile of customers live 6.5 miles away, so all people under that distance will be shown

In [16]:
px.histogram(out_remove,x='distance_from_store',title='Distance from store count histogram minus outliers') #Plotting a histogram of the distances from store with outliers cut out

This new framing shows the distribution a bit better. We're still going to use median.

In [17]:
px.histogram(tech_frame_dict['customer_details'],x='gender',title='Gender distribution') #plotting gender distibution

This one's up for debate as to whether or not I should fill it, but since there are more females than males it's a fine assumption that the few that are unlabeled are female. It shouldn't hurt much.

In [18]:
px.histogram(tech_frame_dict['customer_details'],x='credit_score',title='credit score distribution') #plotting credit score distribution

This credit score distribution shows a normal distribution. As such the mean of the scores can safely be used to fill the values, but I choose to use the median in the code below. This is fine too, because there does not appear to be much skew in the data, so the median should be about equal to the mean. This is done so the code can be a bit more general.

In [19]:
omit = []   # I fill this empty list with the columns that I mentino will be omitted above, but I don't use, but I suppose it's still good to have for refence

for key in nan_dict:              #For all the dataframes
  for col in nan_dict[key].index: #For all of the columns
    
    val = nan_dict[key][col]      #take the fraction of null values

    if  val == 0:                 #if none are null we don't need to do anything
      continue

    elif val < .5:                #If there are a few null values, less than 50% of them, go into filling them

      if col in cat_list:
        tech_frame_dict[key][col].fillna(value=tech_frame_dict[key][col].mode()[0],inplace =True)   #If it's categorical we'll use the mode, Female as discussed above

      else:

        tech_frame_dict[key][col] = tech_frame_dict[key][col].apply(pd.to_numeric, errors='coerce') #There was a weird error with the datatypes, so I had to change the values to numeric with help from online
        tech_frame_dict[key][col].fillna(tech_frame_dict[key][col].median(),inplace =True)          #If it's continuous we'll fill it with the median as discussed above

    else:
      omit.append(col)        #Add to my list to omit if there are too many null values, more than 50%

## Part 2: Data Preparation & Modeling

Use this section to prepare the data and build your model for the task.

### Preprocessing


Now In this section I start combining the dataframes in preparation for splitting into training and testing data. The transaction dataframe also gets summary values pulled from it here, as I can not feed the data in as is. 

In [20]:
#merge the transaction data with the product area data, because those profit margins are useful. This is why it's fine that the product area data is "high dimensional", it's not actually being used as is
full_transactions = tech_frame_dict['transactions'].merge(tech_frame_dict['product_areas'], on = 'product_area_id').dropna(axis='columns')

In [21]:
#merge current customer data together, not transactions yet, since there are multiple for each customer
full_cust = tech_frame_dict['customer_details'].merge(tech_frame_dict['campaign_data'], on = 'customer_id',sort=True).dropna(axis='columns')

Here is probably the trickiest part of the whole project. The transaction data in it's current form is unsuitable for feeding into a lasso regression.

The issues are:

* Multiple inputs for each customer
  *   Since the mutiple rows for each customer would have different info in them
  lasso would end up predicting different scores for the same customer, which wouldn't make sense.

* Date data won't be interpreted properly
 *   By the end of all the processing every individual date would be seen as a different column, so info on what happened in the same month might be lost, though same day would be fine. All of the data is from the same year, so that doesn't really matter

 The ways to solve these problems are to summarize all of the transactions about every customer into one set of values for each customer, then to also summarize data from different months for every customer and add those as new columns.

 In the below code, through intuition, I decide to summarize the transaction data for each customer into these values:


*   Total amount spent by customer
*   Total amount that store has profited off customer
*   Total number of visits customer has made to store
*   Monthly amount spent by customer
*   Monthly number of visits
*   avg number of days between visits

While there are more summary values that could be pulled out for each customer (e.g. what percentage of visits they're buying fruits, average months between visits, etc.) I believe these paint a good picture of the transaction habits of the customers.

In [22]:
#Make some preparations for dealing with transaction data
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
date_dict = dict(zip(range(1,13),months))
cust_groups = full_transactions.groupby(['customer_id'])    #Here we group the transaction data by customer id. Now there a bunch of dataframes with only one customers transaction info in them each.

In [23]:
transaction_summaries_dict = {} #Create dictionary to fill, associating summary values with customer

for name,cust in cust_groups:       #Going through every customers transaction dataframe

  transaction_summaries_dict[name]={}     #Creating a slot for every customer in our dictionary we're creating
  
  cust_copy = cust                                                      #Copying the customers transaction dataframe so we can make changes without changing the original
  cust['profits'] = cust_copy['sales_cost']*cust_copy['profit_margin']  #Calculate how much store has profited in each transaction, creating a new column for it in the created data frame

  total_items_sales_profits = tuple(cust_copy.agg({'num_items':[sum],'sales_cost':[sum], 'profits':[sum]}).values[0])    #Gather total number of items bought, total money spent, total profits for a single customer in a tuple
  total_visits = len(cust)      #Calculate total number of visits for a single customer

  transaction_summaries_dict[name]['total' + ' ' +'num_items' ] = total_items_sales_profits[0]    #Put total items bought data for customer into dictionary
  transaction_summaries_dict[name]['total' + ' ' +'sales' ] = total_items_sales_profits[1]        #Put total sales data for customer into dictionary
  transaction_summaries_dict[name]['total' + ' ' +'visits' ] = total_visits                       #Put total number of visits data for customer into dictionary

  transaction_summaries_dict[name]['total' + ' ' +'profits' ] = total_items_sales_profits[2]      #Put total profit data for customer into dictionary

  transaction_summaries_dict[name]['avg days between visits'] = np.mean((cust.shift()['transaction_date'].dt.day-cust['transaction_date'].dt.day).abs())  #Calculate average days between visits for each customer and plug into dictionary


  cust_month =cust.groupby(cust['transaction_date'].dt.month)    #Create a frame grouping just by months

  month_items_sales_visits_df = cust_month.agg({'num_items':[sum], 'sales_cost':[sum,len]}) #Gather monthly sums of number of items bought and sales costs

  for i,row in month_items_sales_visits_df.iterrows(): #Go through all summary values for customer by month

    mon = date_dict[i]            #Figure out what month this transaction took place

    items = row['num_items']['sum']   #Find total items bought in month
    cost = row['sales_cost']['sum']    #Find total costs bought in month
    visits = row['sales_cost']['len']   #Find total visits in month

    transaction_summaries_dict[name][mon + ' ' +'num_items' ] = items   #Put number of items for month into dictionary
    transaction_summaries_dict[name][mon + ' ' +'sales' ] = cost        #Put sales for month into dictionary
    transaction_summaries_dict[name][mon + ' ' +'visits' ] = visits     #Put number of visits for month into dictionary

Now that we have found the summary metrics and plugged them into a dictionary, pandas lets us easily turn this into a dataframe, which is done below. Since some customers didn't visit certain months the values for thise months are filled with 0s in the created dataframe

In [24]:
transaction_summaries = pd.DataFrame(transaction_summaries_dict).transpose().fillna(value=0) # A dataframe of summary transaction data for each customer to merge with other customer data

Now that we have all of the data in a useable form we can combine everything into one big dataframe

In [25]:
All_data = full_cust.merge(transaction_summaries,left_on = 'customer_id', right_index=True) #Merge all data that will be used for prediction now

In [26]:
new_cat_list = list(filter(lambda col: col in All_data.columns,cat_list)) #There was some sort of issue after going back and editing stuff, but this fixed it

Now we need to separate the data for preprocessing. Categorical values need to be encoded, since the lasso regression needs numbers to work with.

In [27]:
categ = All_data[new_cat_list]
contin = All_data.drop(new_cat_list,axis=1) #Separating categorical and continuous for preprocessing

In [28]:
 #Can't one hot encode with the current datatypes so turning everything into strings and integers
categ = categ.convert_dtypes()

In [29]:
categ.dtypes

gender                   string
campaign_name            string
campaign_date    datetime64[ns]
mailer_type              string
signup_flag               Int64
dtype: object

Now all of the categorical values are encoded

In [30]:
enc = OneHotEncoder().fit(categ) #Create encoder
enc_names = enc.get_feature_names() #Find column names encoder will give
cat_encoded = pd.DataFrame(enc.transform(categ).toarray(),columns = enc_names)  #Create an new dataframe from the categorical values, now encoded, with column names

Now we recombine the newly encoded values with the ones that didn't need it

In [31]:
encoded_all = contin.merge(cat_encoded,left_index=True,right_index=True).convert_dtypes() #Combining dataframes and converting data types so there are no issues.

In [32]:
encoded_all.dtypes #Just looking to see all datatypes are right

customer_id                           Int64
distance_from_store                 float64
credit_score                        float64
total num_items                       Int64
total sales                         float64
total visits                          Int64
total profits                       float64
avg days between visits             float64
Apr num_items                         Int64
Apr sales                           float64
Apr visits                            Int64
Jun num_items                         Int64
Jun sales                           float64
Jun visits                            Int64
Jul num_items                         Int64
Jul sales                           float64
Jul visits                            Int64
Aug num_items                         Int64
Aug sales                           float64
Aug visits                            Int64
Sep num_items                         Int64
Sep sales                           float64
Sep visits                      

Now we scale down the values. The average of every column is taken, then that number is subtracted away from every value, and every value is divided by standard deviation. Essentially every value is replaced with it's z score. This is necessary because the different columns are on different scale (e.g. the credit scores are between 0 and 1, while the distance from stores go all the way up to 400, but mostly between 1 and 7). Because of this difference in scale, the bigger numbers may be seen as "more important" even though what we're really interested in is the variance of the values. Scaling the data combats this, and is done easily with scikitlearns scaler.

In [33]:
pre_scale = encoded_all.drop('customer_id',axis=1) #pull out ids, because those shouldn't be scaled

In [34]:
scaler = StandardScaler().fit(pre_scale)  #Create scaler
all_scaled = pd.DataFrame(scaler.transform(pre_scale),columns=pre_scale.columns)  #Create a dataframe scaling the old dataframe

In [35]:
all_scaled_ids = pd.DataFrame(encoded_all['customer_id']).merge(all_scaled,left_index=True,right_index=True) #Put the ids back on so they're identifiable

This data set has labels (loyalty scores) for half of the customers, so these are the only ones we'll be able to do supervised learning with. Because of this we need to separate out these labeled customers into a separate dataframe for ML modeling.

In [36]:
tech_frame_dict['loyalty_scores']['customer_id']=tech_frame_dict['loyalty_scores']['customer_id'].astype(int) #Can't merge frames between int and float, so switching to int

In [37]:
All_labeled = all_scaled_ids.merge(tech_frame_dict['loyalty_scores'],how='right',on='customer_id',sort=True)  #All labeled customers

Now we pull the labels off of these customers, so models can be trained. If the models were able to see the labels it would be a no brainer because the model would just say "oh, so this is the answer" and spit it right back. We seek to predict the desired value when the computer can not see it, so we need to pull it off. The customer id is also pulled off, because that should not be used to predict, as it's an arbitrary value assigned to a person and should have nothing to do with their loyalty. We set aside the labels for the computer to compare how good it's predictions were and for the computer to know what it's striving for during training.

In [38]:
All_X=All_labeled.drop(['customer_loyalty_score','customer_id'],axis=1)
All_y=All_labeled['customer_loyalty_score']

### Test train split

Now we split the labeled data into test and training sets. The training set will be used to try to figure out what values can be used to predict the outcome and how, and the test is used to see how good our model is.

In [39]:
X_train,X_test,y_train,y_test = train_test_split(All_X,All_y,test_size=.2,random_state=987) #Split data into test and training

### Model training and prediction

Now we fit our actual model

The model I choose to use is Lasso regression. The resons for this are as follows:

*   This is a regression task
  *  Regression is when we seek to predict a specific number based on inputs. Certain models are good for this, and this is one of them.

*   We are not sure if all of our features are important
  *  Some regression models just assume that all of the features are required and use them as such. Their are other methods for choosing important features, but they can be computationally expensive. Lasso combines regression and feature selection in an easy and explainable way.

  In general lasso works by adding a penalty to something called "ordinary least squares regression" or ols. ols just seeks to fit a line through the data as close as possible to all of the data points (loyalty scores). The "penalty" that lasso is based on how many features the model is using to make predictions. So lasso then tries to fit the line as close as possible without using as much data to get it. The tradeoff of how many features should be used to how good the estimate is is optimized mathematically. A sepaate model called ridge regression also does this, but lasso is slightly different in that features get completely eliminated when they're unimportant, while ridge keeps them and just says they're very weak.

  In this case specifically I use a special version of lasso that uses cross validation to find the result. Lasso needs to be told a value, alpha, for how much to penalize. In order to determine this, cross validation does  all of the math multiple times, using different alphas and different combinations of data, then optimizes mathematically and chooses which alpha is best for producing the most accuracy

In [40]:
reg = linear_model.LassoCV(cv=5,random_state=654,max_iter=10000).fit(X_train,y_train) #Fitting lasso model

In [41]:
alphas,coefs,dual_gaps=linear_model.lasso_path(X_train,y_train)   #Pulling some data to show how features were chosen

### Evaluation/ Results

Now that the modeling has been done we will explore the results and see how accurate it was.

Here I plot the Lasso's process in eliminating features. As mentioned earlier it iterates through penalization/ alpha values, and sets the coefficient for the feature equal to 0 when it's unimportant. Higher alphas penalize more, so all of the coefficients are at 0, then as alpha is decreased more values are considered. The higher the absolute value of the coefficient the more important the feature is.

In [42]:
fig = go.Figure() #Create an empty figure

for i in range(len(coefs)):
  fig.add_scatter(x=alphas,y=coefs[i],name=X_train.columns[i])      #Add in the values for the other features

fig.update_layout(    
    title= 'Feature elimination over iterations of alpha',
    xaxis_title="Alpha value",                                  #Add axis labels
    yaxis_title="coefficient value")

The above plot is interactive and can be zoomed in on to explore, but some things that can be discerned are that total visits and the gender female are quickly identified as important, and sitance from store shoots up in importance as well at a certain point. For comparison of what is used for the actual model, the alpha value of .0069 used for it is shown below. The graph comes from a different function than the actual model, so the exact values aren't the same, but it gets the idea across. The actual model made a bunch of these and averaged them to find the best. For further exploration of important features I create bar graphs below.

In [43]:
reg.alpha_  #Showing chosen alpha value

0.0068924549181753345

This bar graph shows the coefficient values chosen for every feature. A streamlined version is below it, showing only those that aren't 0.

In [44]:
coefficients = pd.DataFrame(zip(reg.coef_,X_train.columns),columns=['coef','predictor'])  #Putting together coefficients with columns
px.bar(coefficients,x='predictor',y='coef', title='all feature coefficients') #Plotting coefficients w.r.t. the columns

In [45]:
chosen = coefficients[coefficients['coef'] != 0]      #Finding only columns chosen
px.bar(chosen,x='predictor',y='coef',title='chosen feature coefficients') #Plotting coefficients of chosen columns

From this bar graph we see that the 2 highest magnitude features are x0_F, the encoded indicator showing that the customer is female, and total_visits, the column with the total amount of visits the person made to the store.  The other features are much less important, as the bars are lower. The positive and negative values indicate the relationship, so since total visits is higher that means the more visits you have, the greater your loyalty. Conversely, the fact that you're a woman then makes you less loyal.

Now I take a look at some of the metrics evaluating how good this model is at predicting loyalty

The next 2 cells of code give the R squared value of data and predictions. For the training data it's .68 and for the test it's .57. A score closer to 1 is better, and clearly the training value is better than the test. R squared gives an indication of how well correlated the preidctors are to the outcome, and the training set has a better value by design. The model basically maxed out this value for training, while the test is then used to see how well this works in practice. This value of .57 means there's some fairly good correlation. This will be shown visually momentarily.


In [46]:
reg.score(X_train,y_train)  #Training R squared

0.6754662291505371

In [47]:
reg.score(X_test,y_test)  #Test R squared

0.5676501407309366

Below I plot the actual values against the predicted values. Doing this allows us to see how closely the predictions match. If it was perfect the graph would be a diagonal line from bottom left to top right. The closer it appears to this the better.

In [48]:
y_hat = reg.predict(X_test) #Predict test set from model
#Put together actual and predicted values for easy comparison
score_compare = pd.DataFrame(zip(All_labeled['customer_id'],pd.Series(y_hat),pd.Series(y_test)),columns=['customer_id','predicted','actual'])
#Associating customer ids for the sake of it
check = score_compare.merge(All_labeled,how='left',on='customer_id')

In [49]:
px.scatter(check,x='actual',y='predicted',title='Visual representation of accuracy ')

As can be seen from the graph above, the values actually match pretty well. Something else interesting though is the fact that there seem to be groupings of dots, stratified. The model basically won't predict someone as .2 or .6. As this model is it basically predicts either low, medium, or high, then fine tunes that based on some other values. This can be used later.

For the last bit of evaluation we calculate the root mean squared error of this prediction. This value basically tells us the average amount we can expect to be wrong on any given prediction. As seen below the amount is about .18. On a scale of 0 to 1 for loyalty scores this is actually pretty high, but can still be managed.

In [50]:
mean_squared_error(y_test,y_hat,squared=False)

0.18014455734360837

## Part 3: Application

Use this section to talk about the application of your model. How can the business understand and leverage the outcome of your work? 

First thing's first we can predict the unlabeled people


In [51]:
Unlabeled =  all_scaled_ids.set_index(keys='customer_id').drop(labels=All_labeled['customer_id'])
Unlabeled_y_hat = reg.predict(Unlabeled)
Unlabeled_y_hat

array([ 0.35490319,  0.78975558,  1.06601077,  0.71805428,  0.42658095,
        0.4541319 ,  0.76375963,  0.42993561,  0.68432397,  0.41278162,
        0.82257091,  0.71174405,  0.69637373,  0.37167784,  0.67437337,
        0.15757075,  0.12680058,  0.15525248,  0.44326802,  0.69809795,
        0.99409081,  0.72039906,  0.49508717,  0.33240811,  0.47695818,
        0.82978077,  0.64457111,  0.32803434,  0.29125281,  0.14207947,
        0.10114631,  0.32307432,  0.15680235,  0.65855418,  0.79228116,
        0.72097207,  0.34292429,  0.7220239 ,  0.50873079,  0.46702971,
        0.37060354,  0.42489846,  0.44223328,  0.1494926 ,  0.38077304,
        0.79105931,  0.77421686,  0.15236914,  0.78412687,  0.48095706,
        0.67439441,  0.4078477 ,  0.68698545,  0.83018915,  0.15394249,
        0.11790386,  0.38744684,  0.44754637,  0.3064854 ,  0.39828499,
        0.77607209,  0.40674745,  0.14595183,  0.39070184,  0.35479446,
        0.97567975,  0.43706769,  0.31067627,  0.36990014,  0.40

While this prediction is somewhat good and we could be done here what might be more useful is to group our predictions into loyalty categories rather than loyalty scores. Kmeans clustering is used to make clusters, and a loyalty cluster can then be discerned 

In [52]:
kmeans = KMeans(n_clusters=3, random_state=54).fit(y_hat.reshape(-1,1))   #Perform kmeans clustering


In [53]:
y_hat_clust = kmeans.labels_                                    #Find cluster label of predictions
y_test_clust = kmeans.predict(np.array(y_test).reshape(-1,1))   #Find cluster label of actual values

In [54]:
match_dict={True:1,False:0} #For seeing if cluster matches actual
#Put together prediction, actual, and whether or not the clusters match together into one frame
clust_frame = pd.DataFrame(zip(y_hat,y_test,list(map(lambda ele: match_dict[ele],list(y_test_clust==y_hat_clust)))),columns = ['pred_score','act_score','cluster_match'])

Here is the same plot comparing accuracy from before, but now the points are colored based on if the cluster was identified properly.

In [55]:
px.scatter(clust_frame,x='act_score',y='pred_score',color='cluster_match',title='cluster identified prediction comparison')  

As can be seen from this we can correctly cluster a good portion of the customers like this. Every yellow spot is correctly matched, while every blue spot is incorrect. This was found using 3 clusters on a low, medium, high loyalty basis, but if you wanted to lower it to 2 clusters in a more regular vs. premium member type deal you could get a bit more accurate. 

This confusion matrix shows actual values (columns) against predicted values (rows). The more numbers on the diagonal, the better.

In [56]:
clust_confus = pd.crosstab(y_hat_clust,y_test_clust)    #Create a confusion matrix 
clust_confus

col_0,0,1,2
row_0,,,
0,23,7,0
1,9,21,10
2,0,0,10


The values below show how good the categorization is for each cluster, 0 being high loyalty and 2 being low loyalty. As can be seen we correctly identify about 72% percent of the high loyalty members correctly, and about 75% of the mid tiers correctly. This ought to be good enough for giving out some coupons to the mid and high tier people.

In [57]:
np.diag(clust_confus)/clust_confus.apply(sum,axis=0)  #Calculate accuracies

col_0
0    0.71875
1    0.75000
2    0.50000
dtype: float64

Here are the clusters identified for each of the unlabeled people, not identified by the consulting company.

In [58]:
kmeans.predict(Unlabeled_y_hat.reshape(-1,1)) #Show predicted groups of unknowns

array([1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 2, 2, 2, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 2, 2, 1, 2, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 2,
       1, 0, 0, 2, 0, 1, 0, 1, 0, 0, 2, 2, 1, 1, 1, 1, 0, 1, 2, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 2, 1, 2, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 2, 1, 0, 1, 0, 2, 2, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 2, 1, 1, 2, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 2, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 2, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 2, 1, 0, 2, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 2, 2, 2, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 0, 2, 1, 1, 1,
       0, 0, 2, 0, 2, 1, 1, 2, 1, 2, 0, 1, 0, 1, 0, 1, 1, 0, 2, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 2, 0, 1, 0,

## Wrap Up

* Ensure the notebook is well documented and logical.
* Consider using effective visualizations for the eda, modeling and application components.
* Run all the cells in this notebook to make sure it all works, as expected.
* Please submit your work, inclusive of all relevant assets, in a zip file with your name to the following email address adiamond@rich.com

**© 2021 Rich Products Corporation. All Rights Reserved**